In [1]:
## get ligand as chain L

## this is for first rename some atoms that were
## wrong parsed, and pdb format is not able to recognize them
## basically just to give them some space

import os

pdbs = 'ligands'
for pdb_file in os.listdir(pdbs):
    with open(os.path.join(pdbs, pdb_file), 'r+') as f:
        output = f.read()
        modified_lines = []
        for line in output.split("\n"):
            if "LIG" in line and "M" in line:
                line = line.replace("LIG M", "LIG L",-1)
                if "SYSTNA" in line:
                    line = line.replace("SYSTNA", "SYST N")
                if "SYSTBE" in line:
                    line = line.replace("SYSTBE", "SYST C")
                if "SYSTAL" in line:
                    line = line.replace("SYSTAL", "SYST C")
                if "SYSTCA" in line:
                    line = line.replace("SYSTCA", "SYST C")
                if "SYSTCU" in line:
                    line = line.replace("SYSTCU", "SYST C")
                if "SYST B" in line:
                    line = line.replace("SYST B", "SYST C")
                if "SYSTCS" in line:
                     line = line.replace("SYSTCS", "SYST C")
                if "SYSTCL" in line:
                    line = line.replace("SYSTCL", "SYST CL")
            modified_lines.append(line)
        f.seek(0)  # move the file pointer to the beginning of the file
        f.write("\n".join(modified_lines))  # write the modified lines back to the file
        f.truncate()  # truncate any remaining content if the new content is shorter


In [2]:
import os
from pymol import cmd


## this is for extract ligand PLAS 5K ligand label is L
## this was used here to extract ligands
## it worked, but 2 hundred structures were not able
## to be save it, since they were peptide origine
## and they was not label as L

# Create the output folder if it doesn't exist
output_folder = 'ligands' 
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Loop over all PDB files in the input folder and extract chain L
input_folder = 'ligands' ## same output as input since those pdbs were already without target
for pdb_file in os.listdir(input_folder):
    # Skip non-PDB files
    if not pdb_file.endswith('.pdb'):
        continue
    # Load the PDB file into PyMOL
    cmd.load(os.path.join(input_folder, pdb_file), 'structure')
    # Select and extract chain L
    cmd.select('ligand', 'chain L')
    cmd.save(os.path.join(output_folder, pdb_file), 'ligand')
    # Delete the loaded structure to free up memory
    cmd.delete('structure')


In [4]:
import os
import pandas as pd
import re
import requests

lig = pd.DataFrame()
lig['Drug_ID'] = os.listdir('ligands')

url = 'https://cactus.nci.nih.gov/cgi-bin/translate.tcl'

files = os.listdir('ligands')
smiles_list = []  # Create an empty list to store the SMILES strings

for x in files:
    with open(r"ligands/" + str(x), "rb") as file:
        pdbfile = {'file': file}
        data = {
            "smiles": "",
            "format": "screen",
            "astyle": "kekule",
            "dim": "2D"
        }
        r = requests.post(url, files=pdbfile, data=data)
        tag = "B"
        reg_str = "<" + tag + ">(.*?)</" + tag + ">"
        res = re.findall(reg_str, r.text)
        if res:  # Check if any SMILES string is found
            smiles_list.append(res[0])  # Append the first SMILES string to the list
        else:
            smiles_list.append("")  # If no SMILES string is found, append an empty string

lig['Drug'] = smiles_list  # Assign the list of SMILES strings to the 'Drug' column

# Print the resulting DataFrame
lig

,Drug_ID,Drug
0,4nuc.pdb,CC1=CC(=CC(=C1O)C)N=NC2=CC=C(C=C2)[S](=O)(=O)N...
1,5fp6.pdb,CC1=CC=C(C)C2=C1[NH]C=C2\C=C\CO
2,3kcz.pdb,NC(=O)C1=CC=CC(=C1)N
3,4j33.pdb,CC1=CC2=NC3=C(NC(=O)NC3=O)[N+](=C2C=C1C)CC(O)C...
4,1rlj.pdb,CC1=C(C)C=C2C(=C1)N=C3C(=[NH+]C(=[NH+]C3=[N+]2...
...,...,...
4995,2fz8.pdb,OC(=O)CC1=C2C=CC=CC2=C([O-])[N+](=N1)CC3=NC4=C...
4996,5nlk.pdb,CCOC1=CC=C(C=C1NC(=O)C2=CC(=CC(=C2)NC(C)=O)NC(...
4997,2wkz.pdb,COC(=O)NC(C(=O)NN(CCC(O)(CC1=CC=CC=C1)C(=O)NC2...
4998,6gip.pdb,CC1=NC2=C(C(=C(C=C2)C3=C4C=CC=CC4=NC=C3)C)C(=[...


In [6]:
lig.to_csv('plas5k_lig.tsv', sep='\t')